# Import dependencies

In [1]:
import os

import numpy as np
import pandas as pd

import gzip
import shutil
import json

from ast import literal_eval

from datetime import datetime
from sqlalchemy import create_engine

In [2]:
for i in os.listdir():
    if 'json' in i:
        with gzip.open(i, 'rb') as inputgz_file:
            with open(i.replace('.gz',''), 'wb') as decompressed_file:
                shutil.copyfileobj(inputgz_file, decompressed_file)
     

In [3]:
# read data using pandas func

receipts = pd.read_json('receipts.json',lines=True)
users = pd.read_json('users.json',lines=True)
brands = pd.read_json('brands.json',lines=True)

# Data Exploration

### Receipts data

In [4]:
receipts.shape

(1119, 15)

In [5]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [6]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [7]:
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],
                                  errors='ignore',
                                  record_prefix='rewardsReceiptItemList')\
                    .add_prefix('rewardsReceiptItemList.')

In [8]:
print(receipts_norm.shape)
receipts_norm.head()

(7381, 34)


,rewardsReceiptItemList.barcode,rewardsReceiptItemList.description,rewardsReceiptItemList.finalPrice,rewardsReceiptItemList.itemPrice,rewardsReceiptItemList.needsFetchReview,rewardsReceiptItemList.partnerItemId,rewardsReceiptItemList.preventTargetGapPoints,rewardsReceiptItemList.quantityPurchased,rewardsReceiptItemList.userFlaggedBarcode,rewardsReceiptItemList.userFlaggedNewItem,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
receipts_final = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')
receipts_final.shape

(7381, 50)

In [10]:
receipts_final.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# receipts_final.columns

In [12]:
def date_format(x):
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)


In [13]:
receipts_final['_id'] = receipts_final['_id'].apply(lambda x: x['$oid'])

date_columns = ['createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate']

for col in date_columns:
    receipts_final[col] = receipts_final[col].apply(lambda x: date_format(x))


In [14]:
receipts_final.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Brands Data

In [15]:
brands.shape

(1167, 8)

In [16]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [17]:
brands_norm = pd.json_normalize(brands['cpg'],
                                  errors='ignore',
                                  record_prefix='cpg').add_prefix('cpg.')

In [18]:
brands_norm.head()

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7


In [19]:
brands_norm['cpg.$ref'].value_counts()

brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])

In [20]:
brands_final.shape

(1167, 10)

### Users Data

In [21]:
users.shape

(495, 7)

In [22]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [23]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])

date_columns = ['createdDate',
                'lastLogin']

for col in date_columns:
    users[col] = users[col].apply(lambda x: date_format(x))

In [24]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


# Data Querying

In [25]:
# creating sqlite connection using sqlalchemy
sql = create_engine('sqlite://', echo=False)

In [26]:
users.drop_duplicates(subset=['_id']).to_sql('users',con=sql)

receipts_final.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts',con=sql)

brands_final.drop_duplicates(subset=['brandCode']).drop(columns=['cpg'],axis=1).to_sql('brands',con=sql)

In [27]:
receipts['rewardsReceiptStatus'].value_counts()

FINISHED     5920
FLAGGED       810
SUBMITTED     434
REJECTED      167
PENDING        50
Name: rewardsReceiptStatus, dtype: int64

SQL Query for: 
* When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
* When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

NOTE: We do not have "ACCEPTED" as any entry in receipts data. Therefore assumption here is "FINISHED" status means "ACCEPTED"

In [28]:
from sqlalchemy import text

query1 = text('''
                
                SELECT 
                    T1.avgtotalspent_fin - T2.avgtotalspent_rej AS avgttotalspent_diff,
                    T1.totalitemspurchased_fin - T2.totalitemspurchased_rej AS totalitemspurchased_diff
                FROM
                    (SELECT 
                         AVG(totalSpent) AS avgtotalspent_fin, 
                         SUM(purchasedItemCount) AS totalitemspurchased_fin
                     FROM 
                         receipts            
                     WHERE 
                         rewardsReceiptStatus = 'FINISHED') AS T1
                JOIN
                    (SELECT  
                         AVG(totalSpent) AS avgtotalspent_rej, 
                         SUM(purchasedItemCount) AS totalitemspurchased_rej
                     FROM 
                         receipts            
                     WHERE 
                         rewardsReceiptStatus = 'REJECTED') AS T2
                ON 
                    1 = 1;
                    
                    ''')


result = sql.execute(query1)

print(list(result.keys()))
print(result.fetchall())

['avgttotalspent_diff', 'totalitemspurchased_diff']
[(1224.827964061748, 1364258.0)]


We can see that the difference between average_total_spent for "FINISHED" and "REJECTED" is positive which indicates users have spent from "FINISHED" receipts is higher. One can also noe the difference in amounts using the same query.

Likewise, the total_items_purchased is higher for "FINISHED" receipts than "REJECTED" receipts.

# Data quality check

### We check for pecentage of missing values in all the tables we have

In [29]:
receipts_final.isna().sum() / len(receipts_final) * 100

index                                                         0.000000
_id                                                           0.000000
bonusPointsEarned                                            18.981168
bonusPointsEarnedReason                                      18.981168
createDate                                                    0.000000
dateScanned                                                   0.000000
finishedDate                                                 19.116651
modifyDate                                                    0.000000
pointsAwardedDate                                            17.626338
pointsEarned                                                 15.282482
purchaseDate                                                  6.205121
purchasedItemCount                                            6.557377
rewardsReceiptItemList                                        0.000000
rewardsReceiptStatus                                          0.000000
totalS

* For receipts data we observe that there is significant absence of MetaBrite items information.
* UserFlagged information is absent 95% of times

In [30]:
brands_final.isna().sum() / len(brands_final) * 100

_id              0.000000
barcode          0.000000
category        13.281919
categoryCode    55.698372
cpg              0.000000
name             0.000000
topBrand        52.442159
brandCode       20.051414
cpg.$ref         0.000000
cpg.$id.$oid     0.000000
dtype: float64

* brands data does not have TopBrand Indicator and category code more than 50% of times

In [31]:
users.isna().sum() / len(users) * 100

_id              0.000000
active           0.000000
createdDate      0.000000
lastLogin       12.525253
role             0.000000
signUpSource     9.696970
state           11.313131
dtype: float64

* Users data looks quite good
* All the columns have data available for almost 90% instances
* Data availability for login information and user location can be improved

### Check for mapping between Brands and Receipts tables

In [37]:
# number of unique brands in receipts data

brands_in_receipts = (set(receipts_final.dropna(subset=['rewardsReceiptItemList.brandCode'])\
                          ['rewardsReceiptItemList.brandCode']))
len(brands_in_receipts)

227

In [39]:
# Total number of unique brands
all_brands = set(brands_final.dropna(subset=['brandCode'])['brandCode'])
len(all_brands)

897

In [45]:
unknown_brands = brands_in_receipts.difference(all_brands)
print(len(unknown_brands))

186


* Thus we have 186 brands associated with receipts data but not in the brands table. This indicates a data drift and calls for stronger measures to ensure data quality.